In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [6]:
driver = webdriver.Chrome()

In [7]:
driver.get("https://www.selenium.dev/selenium/web/web-form.html")

In [8]:
title = driver.title

In [9]:
print(title)

Web form


In [10]:
driver.implicitly_wait(0.5)

In [11]:
text_box = driver.find_element(By.NAME, "my-text")
submit_button = driver.find_element(By.CSS_SELECTOR, "button")

In [12]:
text_box.send_keys("Hello, world!")
submit_button.click()

In [13]:
message = driver.find_element(By.ID, "message")
print(message.text)

Received!


In [14]:
driver.quit()

In [50]:
from dotenv import load_dotenv
import os
from langchain.chat_models import init_chat_model


load_dotenv() 

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


# model1 = init_chat_model("llama3-8b-8192", model_provider="groq")
llm = init_chat_model("deepseek-r1-distill-llama-70b", model_provider="groq")
llm2 = init_chat_model("llama-3.1-8b-instant", model_provider="groq")
llm3 = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")


ValueError: Unsupported function

('name', 'search_for_flights_and_hotel')

Functions must be passed in as Dict, pydantic.BaseModel, or Callable. If they're a dict they must either be in OpenAI function format or valid JSON schema with top-level 'title' and 'description' keys.

In [51]:
from langchain.tools import tool

In [80]:
from pydantic import BaseModel, Field
from typing_extensions import Literal
from langchain_core.messages import HumanMessage, SystemMessage
class Route(BaseModel):
    step: Literal['search', 'booking'] = Field(
        None, description="The next step in the routing process"
    )


# Augment the LLM with schema for structured output
router = llm.with_structured_output(Route)


In [81]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
# State
class State(TypedDict):
    input: str
    decision: str
    output: str

In [82]:
#Nodes
#Booking node
def llm_call_1(state:State):
    result = booking.invoke(state['input'])
    return {'output': result.content}

In [83]:
#search node
def llm_call_2(state:State):
    result = search.invoke(state['input'])
    return {'output': result.content}


In [84]:
@tool
def search_for_flights_and_hotel(driver):
    '''
    Search for flights and hotels using the provided Selenium WebDriver.
    '''

    driver = webdriver.Chrome()
    driver.get("https://aviasales.tpx.lv/DhO1fN03")

    return llm2.invoke(f'''
                        Use the above webpage to search for flights and hotels.
                        Return the results inform of a list for example:
                        - Flight from A to B on date X
                        - Hotel in location Y
                        ''')

In [85]:
@tool
def make_booking(driver):
    '''
    Make a flight booking using the provided Selenium WebDriver.
    '''
    driver = webdriver.Chrome()
    driver.get("https://aviasales.tpx.lv/DhO1fN03")

    return llm3.invoke(f'''
                        Use the above webpage to make a flight booking.
                        Return a link of the booking details.
                        You may be faced with some challenges when the webpage asks you for confidential information.
                        Feel free to return to the user for more clarity
                        ''')

In [86]:
#LLMs
search_tools = [search_for_flights_and_hotel]
booking_tools = [make_booking]
search = llm2.bind_tools(search_tools)
booking = llm3.bind_tools(booking_tools)

In [87]:
def llm_call_router(state: State):
    '''
    Route the LLM calls to the appropriate node
    '''
    decision = router.invoke(
        [
            SystemMessage(
                content="You are a routing LLM. Based on the user's input, decide whether to search for flights and hotels or make a booking. If the user wants to search, choose 'search'. If the user wants to make a booking, choose 'booking'."
            ),
            HumanMessage(
                content=state["input"]
            )
        ]
    )
    return {"decision": decision.step}

In [88]:
# Conditional edge function to route to the apporopriate node
def route_decision(state: State):
    if state['decision'] == 'search':
        return 'llm_call_1'
    elif state['decision'] == 'booking':
        return 'llm_call_2'

In [89]:
#Build workflow
router_builder = StateGraph(State)

#Add Nodes
router_builder.add_node("llm_call_1", llm_call_1)
router_builder.add_node("llm_call_2", llm_call_2)
router_builder.add_node("llm_call_router", llm_call_router)

#Add edges to connect nodes
router_builder.add_edge(START, "llm_call_router")
router_builder.add_conditional_edges(
    "llm_call_router",
    route_decision,
    {
        "llm_call_1": "llm_call_1",
        "llm_call_2": "llm_call_2"
    },
)

router_builder.add_edge("llm_call_1", END)
router_builder.add_edge("llm_call_2", END)


In [90]:
#Compile workflow
router_workflow = router_builder.compile()

In [91]:
from IPython.display import Image, display

display(Image(router_workflow.get_graph().draw_mermaid_png()))


ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [92]:
#Invoke
state = router_workflow.invoke(
    {
        'input': 'I want to book a flight to New York'
    }
)

In [96]:
print(state["output"])